In [14]:
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Set up Google Sheets API
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(r'C:\Users\danie\credentials.json', scope)
client = gspread.authorize(creds)

# Access the Google Sheet
sheet = client.open("Dynamic Python Graph").sheet1

# Initialize the Dash app
app = Dash(__name__)

# Fetch initial data to set up the slider marks
data = sheet.get_all_values()
header = data.pop(0)  # Separate header from data
months = [row[0] for row in data]

app.layout = html.Div([
    dcc.Slider(
        id='month-slider',
        min=0,
        max=len(months) - 1,
        value=0,
        marks={i: month for i, month in enumerate(months)},
        step=None
    ),
    dcc.Graph(id='speedometer')
])

@app.callback(
    Output('speedometer', 'figure'),
    Input('month-slider', 'value')
)
def update_speedometer(selected_month_index):
    # Fetch the latest data
    data = sheet.get_all_values()
    header = data.pop(0)  # Separate header from data

    # Convert data into a dictionary
    data_dict = {row[0]: float(row[1]) for row in data}
    months = list(data_dict.keys())

    selected_month = months[selected_month_index]
    value = data_dict[selected_month]

    fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=value,
        title={'text': "Speed"},
        gauge={
            'axis': {'range': [0, 200]},
            'bar': {'color': "darkblue"},
            'steps': [
                {'range': [0, 50], 'color': "green"},
                {'range': [50, 100], 'color': "yellow"},
                {'range': [100, 200], 'color': "red"}
            ],
        }
    ))

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)
